# 🏦 Customer Communication Review with Reflection Pattern

## Overview

This notebook demonstrates the **reflection and retry pattern** for financial services communications - ensuring customer-facing responses meet quality and compliance standards before being sent.

### 💼 Industry Use Case: Customer Communication Quality Assurance

When responding to customer inquiries about financial products:
1. **Response Generator** creates initial customer communication
2. **Quality Reviewer** evaluates against compliance and clarity criteria
3. If not approved, Generator revises based on feedback
4. Only approved responses are sent to the customer

### ⚠️ Important Financial Disclaimer
> **This notebook is for educational purposes only.** Financial communications require actual compliance review, legal approval, and adherence to regulatory requirements.

### Key Concepts

| Concept | Description |
|---------|-------------|
| **WorkflowAgent** | Wraps workflow as a standard agent interface |
| **Cyclic Workflow** | Worker ↔ Reviewer bidirectional flow |
| **AgentRunUpdateEvent** | Emits only approved responses |
| **Structured Output** | Pydantic models for review feedback |

### Architecture

```
Customer Question
    ↓
Response Generator (creates communication)
    ↓
Quality Reviewer (evaluates against criteria)
    ↓ (if not approved)
Generator (revises with feedback) ──┐
    ↓                                │
Reviewer (re-evaluates)              │
    └─────────────────────────────────┘
    ↓ (if approved)
Approved Response → Customer
```

## Prerequisites

- ✅ OpenAI API key configured: `OPENAI_API_KEY`
- ✅ Agent Framework installed: `pip install agent-framework`

## 1️⃣ Setup and Imports

In [ ]:
import asyncio
from dataclasses import dataclass
from uuid import uuid4

import os
from dotenv import load_dotenv
from azure.identity import AzureCliCredential

from agent_framework import (
    AgentResponseUpdate,
    AgentRunUpdateEvent,
    ChatClientProtocol,
    ChatMessage,
    Contents,
    Executor,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from pydantic import BaseModel

# Load environment variables from .env file
load_dotenv('../../.env')

# Verify environment is loaded
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
print(f"✅ Environment loaded: {azure_endpoint is not None and deployment_name is not None}")

## 2️⃣ Define Message Types

### ReviewRequest
Request from Generator to Reviewer containing:
- `request_id`: Unique identifier for correlation
- `user_messages`: Original customer question
- `agent_messages`: Generated response for review

### ReviewResponse
Feedback from Reviewer to Generator containing:
- `request_id`: Correlates with original request
- `feedback`: Specific guidance for improvement
- `approved`: Whether response meets all criteria

In [ ]:
@dataclass
class ReviewRequest:
    """Structured request passed from Worker to Reviewer for evaluation."""
    request_id: str
    user_messages: list[ChatMessage]
    agent_messages: list[ChatMessage]


@dataclass
class ReviewResponse:
    """Structured response from Reviewer back to Worker."""
    request_id: str
    feedback: str
    approved: bool

## 3️⃣ Create Quality Reviewer Executor

The Reviewer evaluates customer communications against FSI quality criteria:

### Review Criteria:
1. **Accuracy**: Information is correct and up-to-date
2. **Clarity**: Easy for customers to understand
3. **Compliance**: Includes required disclaimers
4. **Completeness**: Addresses all parts of the question
5. **Professionalism**: Appropriate tone and language

### Implementation:
- Uses Pydantic `BaseModel` with `response_format` for structured feedback
- Includes full conversation context for accurate review
- Returns actionable feedback for revisions

In [ ]:
class QualityReviewer(Executor):
    """Executor that reviews customer communications for quality and compliance."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client

    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext[ReviewResponse]) -> None:
        print(f"📋 Quality Reviewer: Evaluating response {request.request_id[:8]}...")

        # Define structured schema for the LLM to return.
        class _Response(BaseModel):
            feedback: str
            approved: bool

        # Construct FSI-focused review instructions
        messages = [
            ChatMessage(
                role=Role.SYSTEM,
                text=(
                    "You are a Quality Reviewer for customer communications at a financial institution.\n"
                    "Evaluate the response against these criteria:\n"
                    "1. ACCURACY: Information is correct and current\n"
                    "2. CLARITY: Easy for customers to understand, no jargon\n"
                    "3. COMPLIANCE: Includes appropriate disclaimers when needed\n"
                    "4. COMPLETENESS: Addresses all parts of the question\n"
                    "5. PROFESSIONALISM: Appropriate tone, no promises\n\n"
                    "Approve only if ALL criteria are satisfied.\n"
                    "Provide specific, actionable feedback for improvements."
                ),
            )
        ]
        messages.extend(request.user_messages)
        messages.extend(request.agent_messages)
        messages.append(ChatMessage(role=Role.USER, text="Review this customer communication."))

        print("📋 Quality Reviewer: Evaluating against FSI criteria...")
        response = await self._chat_client.get_response(messages=messages, options={"response_format": _Response})

        parsed = _Response.model_validate_json(response.messages[-1].text)

        status = "✅ APPROVED" if parsed.approved else "⚠️ REVISION NEEDED"
        print(f"📋 Quality Reviewer: {status}")
        print(f"📋 Feedback: {parsed.feedback}")

        await ctx.send_message(
            ReviewResponse(request_id=request.request_id, feedback=parsed.feedback, approved=parsed.approved)
        )

## 4️⃣ Create Response Generator Executor

The Generator creates customer communications and revises based on feedback.

### Handler Methods:

#### `handle_user_messages`
- Receives customer questions
- Generates initial response with financial context
- Sends to Quality Reviewer for evaluation

#### `handle_review_response`
- If approved: Emits to customer via `AgentRunUpdateEvent`
- If not approved: Incorporates feedback and regenerates
- Tracks pending requests for correlation

In [ ]:
class ResponseGenerator(Executor):
    """Executor that generates customer communications and incorporates feedback."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client
        self._pending_requests: dict[str, tuple[ReviewRequest, list[ChatMessage]]] = {}

    @handler
    async def handle_user_messages(self, user_messages: list[ChatMessage], ctx: WorkflowContext[ReviewRequest]) -> None:
        print("💬 Generator: Received customer question, creating response...")

        # Initialize with FSI customer service context
        messages = [
            ChatMessage(
                role=Role.SYSTEM, 
                text=(
                    "You are a helpful customer service representative at a retail bank.\n"
                    "When responding to customers:\n"
                    "- Be friendly and professional\n"
                    "- Use clear, simple language\n"
                    "- Include appropriate disclaimers for financial information\n"
                    "- Never make guarantees about rates, approvals, or returns\n"
                    "- Recommend consulting with advisors for specific advice"
                )
            )
        ]
        messages.extend(user_messages)

        print("💬 Generator: Generating response...")
        response = await self._chat_client.get_response(messages=messages)
        print(f"💬 Generator: Response created")

        messages.extend(response.messages)

        # Send for quality review
        request = ReviewRequest(request_id=str(uuid4()), user_messages=user_messages, agent_messages=response.messages)
        print(f"💬 Generator: Sending for review (ID: {request.request_id[:8]})")
        await ctx.send_message(request)

        self._pending_requests[request.request_id] = (request, messages)

    @handler
    async def handle_review_response(self, review: ReviewResponse, ctx: WorkflowContext[ReviewRequest]) -> None:
        print(f"💬 Generator: Review received for {review.request_id[:8]} - Approved: {review.approved}")

        if review.request_id not in self._pending_requests:
            raise ValueError(f"Unknown request ID: {review.request_id}")

        request, messages = self._pending_requests.pop(review.request_id)

        if review.approved:
            print("✅ Generator: Response approved! Sending to customer...")
            contents: list[Contents] = []
            for message in request.agent_messages:
                contents.extend(message.contents)

            # Emit approved result to external consumer via AgentRunUpdateEvent
            await ctx.add_event(
                AgentRunUpdateEvent(self.id, data=AgentResponseUpdate(contents=contents, role=Role.ASSISTANT))
            )
            return

        print(f"⚠️ Generator: Revising based on feedback: {review.feedback}")

        # Incorporate feedback and regenerate
        messages.append(ChatMessage(role=Role.SYSTEM, text=f"Quality Review Feedback: {review.feedback}"))
        messages.append(ChatMessage(role=Role.SYSTEM, text="Please revise the response addressing the feedback."))
        messages.extend(request.user_messages)

        response = await self._chat_client.get_response(messages=messages)
        print("💬 Generator: Revised response created")

        messages.extend(response.messages)

        # Send revised response for re-review
        new_request = ReviewRequest(
            request_id=review.request_id, user_messages=request.user_messages, agent_messages=response.messages
        )
        await ctx.send_message(new_request)

        self._pending_requests[new_request.request_id] = (new_request, messages)

## 5️⃣ Build Workflow and Wrap as Agent

### Workflow Construction:
1. **Create chat client** for both Generator and Reviewer
2. **Create executors** with the chat client
3. **Build cyclic workflow** with bidirectional edges
4. **Wrap as WorkflowAgent** for standard agent interface

In [ ]:
async def run_customer_communication_workflow() -> None:
    print("🏦 CUSTOMER COMMUNICATION QUALITY ASSURANCE")
    print("=" * 60)

    # Create Azure OpenAI chat client
    print("Creating Azure OpenAI chat client...")
    chat_client = AzureOpenAIChatClient(
        credential=AzureCliCredential(),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    )
    print("✅ Azure OpenAI Chat Client created")

    print("Building workflow with Generator ↔ Reviewer cycle...")
    
    # Build workflow using register_executor pattern (matching official sample)
    agent = (
        WorkflowBuilder()
        .register_executor(
            lambda: ResponseGenerator(id="response_generator", chat_client=chat_client),
            name="generator",
        )
        .register_executor(
            lambda: QualityReviewer(id="quality_reviewer", chat_client=chat_client),
            name="reviewer",
        )
        .add_edge("generator", "reviewer")  # Generator sends response to Reviewer
        .add_edge("reviewer", "generator")  # Reviewer provides feedback to Generator
        .set_start_executor("generator")
        .build()
        .as_agent()  # Wrap workflow as an agent
    )
    print("✅ Workflow built with reflection pattern")

    # Sample customer question
    customer_question = """
    I've been with your bank for 5 years and I'm interested in opening a 
    high-yield savings account. Can you explain the current interest rates 
    and any fees associated with the account? Also, is my money FDIC insured?
    """

    print("\n📧 CUSTOMER INQUIRY:")
    print("-" * 60)
    print(customer_question.strip())
    print("-" * 60)
    print("\n⏳ Processing with quality review...\n")

    # Run agent in streaming mode to observe incremental updates
    async for event in agent.run_stream(customer_question):
        print(f"📤 Agent Event: {event}")

    print("\n✅ Customer communication workflow complete!")
    print("\n⚠️ DISCLAIMER: This is a demonstration. Actual customer")
    print("   communications require compliance and legal review.")

## 6️⃣ Run the Customer Communication Workflow

In [ ]:
await run_customer_communication_workflow()

## 📝 Key Takeaways

### Reflection Pattern for FSI

| Benefit | Description |
|---------|-------------|
| **Quality Assurance** | All responses reviewed before sending |
| **Compliance** | Ensure disclaimers and accurate information |
| **Consistency** | Standard quality criteria applied |
| **Audit Trail** | Review feedback logged for analysis |

### Industry Use Cases

- **Customer Communications**: Email, chat, letters
- **Investment Recommendations**: Ensure suitability disclosures
- **Loan Documentation**: Verify required information
- **Regulatory Filings**: Check completeness and accuracy

### Pattern Summary

```python
# 1. Create cyclic workflow
workflow = (
    WorkflowBuilder()
    .add_edge(generator, reviewer)
    .add_edge(reviewer, generator)  # Bidirectional!
    .set_start_executor(generator)
    .build()
)

# 2. Wrap as agent
agent = workflow.as_agent()

# 3. Run - only approved responses emitted
async for event in agent.run_stream(customer_question):
    print(event.text)  # Approved response only
```

### Production Considerations

- **Iteration Limits**: Prevent infinite loops (max 3 retries)
- **Cost Monitoring**: Each iteration costs LLM calls
- **Logging**: Track all review feedback for quality metrics
- **Escalation**: Route repeated rejections to human review